# 1) Utility functions and general imports

In [1]:
import h5py
import numpy as np
import scipy as scpy
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
from scipy import optimize as opt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline


# 2) Creating the functions used throughout the notebook

In [2]:
# Defining utility functions for reading images and data in
def show_image(array, label):
    im = Image.fromarray(array)
    print("Item of label={}".format(label))
    return(imshow(im))

def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()

def precision_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()    

def recall_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns


def calculate_acc_metrics(cm):
    tp = []
    tn = []
    fp = []
    fn = []
        
    for i in range(0):


        tp.append(cm[i][i])
        fp.append(sum(cm[:,i]) - cm[i][i])
        fn.append(sum(cm[i,:]) - cm[i][i])
        tn.append(np.sum(cm) - (tp + fp + fn))

    return(tp)

def create_confusion_matrix(actual, preds):
    
  classes = len(np.unique(actual)) # Number of classes 
  
  cm = np.zeros((classes, classes))

  for i in range(len(actual)):
    cm[actual[i]][preds[i]] += 1
    
  return(cm)   

def shape_matrix_process(split_percent):
    
            
        # Read the source files in from file
        data, label, test_data, test_label = read_data_in()
        
        # Creating a % split of training and validation data for the 
        # training model to be trained on
        indices = range(data.shape[0])
        classes = np.unique(label)
        
        training_records = int(split_percent * data.shape[0])        
        
        # Get the records that are part of the indices declared above
        train_data = data[:training_records]
        validate_data = data[training_records:]        
        
        # Get the labels that are part of the indices declared above
        label_train = label[:training_records]
        label_validate = label[training_records:]        
        
        
        # We can vectorise our array lambda checker to build out the pixel distributions
        #get_class_data = np.vectorize(lambda arr: label==i )
                        
        #for i in classes:
        #    print("Pixel histogram prior to feature scaling : ".format(classes[i]))
        #    print(i)
        # 
        #    d=[]
        #    d = data[get_class_data(i)]
        #    h = plt.hist(d[i].ravel())
        #    plt.show()

        X_train = train_data.reshape(-1, 784)
        X_validate = validate_data.reshape(-1, 784)        
                
        # Change our X_train data set to ensure later normalisation technique doesn't raise warnings on the data type
        X_train = X_train.astype('float32')
        # Change our X_Validate data set to ensure later normalisation technique doesn't raise warnings on the data type
        X_validate = X_validate.astype('float32')
        
        y_train = np.ravel(label_train)
        y_validate = np.ravel(label_validate)                    

               
        ########################
        # Now to transform test#
        ########################                  
        X_test = test_data.reshape(-1,784)
        y_test = test_label [:2000]       
        
        X_test = X_test.astype('float32')
                    
        ##########################################################################  
        # MINMAXSCALER IS THE ONLY SCALER FOUND TO RELIABLY RESULT IN CONVERGENCE*
        ##########################################################################
                    
        # Normalise our training, validation & test-sets
        X_train = normalise(X_train)                                
        X_validate = normalise(X_validate)
        X_test = normalise(X_test)                
        
        print('Total number of records in X train: {}'.format(X_train.shape[0]))
        print('Total number of records in y train: {}'.format(y_train.shape[0]))
        print('Total number of features in X train: {}'.format(X_train.shape[1]))

        print('Total number of records in X train (valdation set): {}'.format(X_validate.shape[0]))
        print('Total number of features in X train (validation set): {}'.format(X_validate.shape[1]))
                

        ##########################################################################  
        # MINMAXSCALER IS THE ONLY SCALER FOUND TO RELIABLY RESULT IN CONVERGENCE*
        # HOWEVER NEEDS FURTHER TESTING =========================================#
        ##########################################################################
   

                                          
        print('Total number of records in X test: {}'.format(X_test.shape[0]))
        print('Total number of features in X test: {}'.format(X_test.shape[1]))
        
        #Return our training, validation & test sets
        return(X_train,X_validate,X_test,y_train,y_validate,y_test)
    
    
# Import the files in from python h5 format
def read_data_in():

    ## Dan's desktop folder location - NEEDS CHANGING
    with h5py.File('../Input//images_training.h5','r') as H:
        data = np.copy(H['data'])
    with h5py.File('../Input/labels_training.h5','r') as H:
        label = np.copy(H['label'])
    with h5py.File('../Input/images_testing.h5','r') as H:
        data_test = np.copy(H['data'])
    with h5py.File('../Input/labels_testing_2000.h5','r') as H:
        label_test = np.copy(H['label'])     
    
    return(data, label, data_test, label_test)

# Is the matrix symmetric?
def is_symmetric(X, tolerance = 1e-9):
    return(np.allclose(X,X.T, atol=tolerance))

def normalise(X):    
    X_scaled = (X - np.min(X,axis=0)) / (np.max(X, axis=0) - (np.min(X,axis=0) ))
    return(X_scaled)


class LogisticRegression:
    def __init__(self,  max_iter, intercept, k, lmbda ):
        #self.lr = lr
        self.max_iter = max_iter
        self.intercept = intercept
        self.k = k
        self.lmbda = lmbda
        print ("Logistic Regression model initialised with Max iterations = {}, K (classes) = {}, lmbda (Regularisation parameter) ={}".format(self.max_iter,self.k, self.lmbda))
    
    def add_intercept(self, X_mat,y_mat):
        m = len(y_mat)
        ones = np.ones((m,1))

        X_mat = np.concatenate((ones,X_mat),axis=1)
        m_shape,n_shape = X_mat.shape
        
        return(X_mat,n_shape)
    
    # Defining the sigmoid function required in LR
    def __sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    
    def fit(self, X_mat,max_iter, y_mat,n, full_outp):
                        
        theta = np.zeros((k,n)) #inital parameters                        
        print("                                            ")
        print("############################################")
        print("### Gradient Descent Optimisation beginning")
        print("############################################")
        for i in range(k):
            label_class = i if i else 0
            print('Class {} being optimised'.format(i))
            theta[i] = opt.fmin_cg(f = self.__cost_func_vectorised, x0 = theta[i].flatten(), gtol = 1e-03, fprime = self.__gradient_reg_vectorised, args = (X_mat,(y_mat == label_class).flatten(), self.lmbda),maxiter= max_iter, disp = True, full_output = full_outp)                                    
        print("###########################################")
        print("### Gradient Descent Optimisation finished")
        print("###########################################")

        return(theta)      
    
    # Vectorised gradient
    def __gradient_reg_vectorised(self, theta, X_arr, y_arr, lmbda):
        m = len(y_arr)
        temp = self.__sigmoid(np.dot(X_arr, theta)) - y_arr
        temp = np.dot(temp.T, X_arr).T / m + theta * lmbda / m
        temp[0] = temp[0] - theta[0] * lmbda / m
        return temp
    
    # Vectorised cost function
    def __cost_func_vectorised(self,theta, X_arr, y_arr, lmbda):
        m = len(y_arr)
        temp_weight1 = np.multiply(y_arr, np.log(self.__sigmoid(np.dot(X_arr, theta))))
        temp_weight2 = np.multiply(1-y_arr, np.log(1-self.__sigmoid(np.dot(X_arr, theta))))
        
        # Printing out cost function value
        #print(np.sum(temp_weight1 + temp_weight2) / (-m) + np.sum(theta[1:]**2) * self.lmbda / (2*m))
        return np.sum(temp_weight1 + temp_weight2) / (-m) + np.sum(theta[1:]**2) * self.lmbda / (2*m)    
    
    def predict(self,X,y,theta):    
        preds = []
        preds = np.argmax(X @ theta.T, axis = 1)
        #preds = X @ theta.T
        preds = [e if e else 0 for e in preds]
        average_pred = np.mean(preds == y.flatten()) * 100        
        return(preds, average_pred)        
        
        
    def final_predict(self,X,theta):    
        preds = []
        preds = np.argmax(X @ theta.T, axis = 1)        
        preds = [e if e else 0 for e in preds]     
        preds = np.asarray(preds)
        return(preds)
    
    def predict_prob(self,X, theta):
        return (self.__sigmoid(np.dot(X),theta.T))
    
        
 

## Class counts 

In [3]:
# How many unique classes are we dealing with? Do we need to perform any sampling for class imbalance?
classes = np.unique(label)
nclasses = len(classes)
test_classes = np.unique(label_test)
test_nclasses = len(test_classes)

print('Total number of classes in Train: ', nclasses)
print('Total number of classes in Test : ', test_nclasses)
print('Classes to classify in Train are : ', classes)
print('Classes to classify in Test are : ', test_classes)


unique, counts = np.unique(label, return_counts=True)
print('Distribution of labels against total population in Train:')
dict(zip(unique,counts))

unique_test, count_test = np.unique(label_test, return_counts=True)
print('Distribution of labels against total population in Test:')
dict(zip(unique_test,count_test))

NameError: name 'label' is not defined

In [ ]:
print("Pixel histogram prior to feature scaling of test data : ".format(classes[i]))
print(i)        
d=[]
d = data_test[get_class_data(1)]
#h = plt.hist(d[i].ravel())
#plt.show()


In [ ]:
get_class_data = np.vectorize(lambda arr: label==i )

fig, a = plt.subplots(5,2, sharex='row', sharey='col')
fig.set_size_inches(20,25)
fig.subplots_adjust(hspace = 0.3, wspace = 0.1)
fig.suptitle('Pixel value - Histogram and distribution', fontsize = 16)

classes = np.unique(label)
a = a.ravel()
        
for i,ax in enumerate(a):
    print("Pixel histogram prior to feature scaling : {} ".format(classes[i],i))
    #print(i)

    #ax=[]
    d = data[get_class_data(i)]
    ax.hist(d[i], bins = 20)
    ax.set_title(i, fontsize = 16)    
        
    
plt.tight_layout    
plt.savefig("PixelHist_Distiribution.png", dpi = 300)
        
        
        


## For reference: https://github.com/zalandoresearch/fashion-mnist



| Label | Description   |
|------|------|
|   0  | T-shirt/top|
|   1  | Trouser|
|   2  | Pullover|
|   3  | Dress|
|   4  | Coat|
|   5  | Sandal|
|   6  | Shirt|
|   7  | Sneaker|
|   8  | Bag|
|   9  | Ankle boot|

# Time to train and predct using LogisticRegression - One vs Rest!
# Hyper-parameter tuning stage

In [4]:
%%time

# Hyper-parameter tuning - uncomment the first value if you want to loop
#lambda_values = [0.1, 0.5, 1, 2.5, 5]
lambda_values = [1]
#split_percent_values []

prediction_average_list = []
prediction_average_validate_list = []
prediction_average_test_list = []    

for i in lambda_values:

    # Read the source files in
    # Ben/Stef - You'll need to change this
    data, label, data_test, label_test = read_data_in()

    # Declare variables for instantiation of the LogisticRegression Class
    #lmbda = 2.5
    k = 10
    intercept = True
    max_iter = 500
    
    data_test = data_test[:2000]

    #########################################################################################
    # Once data is read in from file data, we split it for training, validation and testing #
    # standardise, & re-shape it process                                                    #
    #########################################################################################            
    X_train,X_validate,X_test,y_train,y_validate,y_test = shape_matrix_process(split_percent=0.99)    
    X_test = X_test[:2000]

    ############################################
    # Instantiate our logistic Regression model#
    ############################################
    print('#################################################')
    model = LogisticRegression(max_iter, intercept, k, i)
    print('#################################################')

    # Carry out principal component analysis and project back with 99% of variance
    #X_train, X_reduced  = compress_project(percent=.99,data_to_compress=X_train)

    # Add the intercept value
    X_train,n = model.add_intercept(X_train,y_train)
    X_validate,n = model.add_intercept(X_validate,y_validate)

    # Create a theta matrix to capture values of theta when we minimis the objective function below
    theta = np.zeros((k,n)) #inital parameters

    #############################################################################################
    # Using conjugate gradient, we attempt to carry out optimisation to find theta and therefore#
    # Using theta and the data features to predict each class   #################################
    #############################################################################################
    
    # Do you want to see a verbose/full output for conjugate gradient descent?
    
    #theta = model.fit(X_mat=X_train,max_iter=500,y_mat=y_train,n=n, full_outp=False)
    
    
    #theta, fopt, func_calls, grad_calls,warnflag  = model.fit(X_mat=X_train,max_iter=500,y_mat=y_train,n=n, full_outp=False)
    theta  = model.fit(X_mat=X_train,max_iter=2000,y_mat=y_train,n=n, full_outp=False)
            
    #######################################################################################
    # Carry out the predictions based on the trained model and then carry out predictions #
    # on the unseen test data                                                             #
    #######################################################################################
    X_test,n = model.add_intercept(X_test,y_test)
    
    preds, prediction_average = model.predict(X=X_train,y=y_train,theta=theta)    
    prediction_average_list.append(prediction_average)
        
    preds_validate, prediction_average_validate = model.predict(X=X_validate,y=y_validate,theta=theta)
    prediction_average_validate_list.append(prediction_average_validate)
            
    preds_test,prediction_average_test = model.predict(X_test,y_test,theta=theta)    
    prediction_average_test_list.append(prediction_average_test)
    
    cm_train = create_confusion_matrix(y_train, preds)
    cm = create_confusion_matrix(y_validate, preds_validate)
    
    print("Probability averages on train: ",*prediction_average_list, sep =',')
    print("Probability averages on validation: ",*prediction_average_validate_list, sep =',')    
    #print("precision total - train: ", precision_average(cm_train))    
    #print("precision total - validation: ", precision_average(cm))
    #print("recall total - train: ", recall_average(cm_train))   
    #print("recall total - validation: ", recall_average(cm))   
    

Total number of records in X train: 24000
Total number of records in y train: 24000
Total number of features in X train: 784
Total number of records in X train (valdation set): 6000
Total number of features in X train (validation set): 784
Total number of records in X test: 5000
Total number of features in X test: 784
#################################################
Logistic Regression model initialised with Max iterations = 500, K (classes) = 10, lmbda (Regularisation parameter) =1
#################################################
                                            
############################################
### Gradient Descent Optimisation beginning
############################################
Class 0 being optimised
Optimization terminated successfully.
         Current function value: 0.102454
         Iterations: 26
         Function evaluations: 52
         Gradient evaluations: 52
Class 1 being optimised
Optimization terminated successfully.
         Current functio

# Final model execution - Based on hyper-parameter tuning from above 

In [6]:
%%time

# Declare the result arrays
prediction_average_list = []
prediction_average_test_list = []    

data, label, data_test, label_test = read_data_in()

# Declare variables for instantiation of the LogisticRegression Class
lmbda = 1
k = 10
intercept = True
max_iter = 500


#X_test = data_test[:2000]

#########################################################################################
# Once data is read in from file data, we split it for training, validation and testing #
# standardise, & re-shape it process                                                    #
#########################################################################################            
X_train,X_validate,X_test,y_train,y_validate,y_test = shape_matrix_process(split_percent=0.99)    

# Saving for later final predictions
final_data_test = X_test
X_test = X_test[:2000]


############################################
# Instantiate our logistic Regression model#
############################################
print('#################################################')
model = LogisticRegression(max_iter, intercept, k, lmbda)
print('#################################################')

# Carry out principal component analysis and project back with 99% of variance
#X_train, X_reduced  = compress_project(percent=.99,data_to_compress=X_train)

# Add the intercept value
X_train,n = model.add_intercept(X_train,y_train)
X_validate,n = model.add_intercept(X_validate,y_validate)

# Create a theta matrix to capture values of theta when we minimis the objective function below
theta = np.zeros((k,n)) #inital parameters

#############################################################################################
# Using conjugate gradient, we attempt to carry out optimisation to find theta and therefore#
# Using theta and the data features to predict each class   #################################
#############################################################################################

# Do you want to see a verbose/full output for conjugate gradient descent?
theta  = model.fit(X_mat=X_train,max_iter=500,y_mat=y_train,n=n, full_outp=False)

#######################################################################################
# Carry out the predictions based on the trained model and then carry out predictions #
# on the unseen test data                                                             #
#######################################################################################
X_test,n = model.add_intercept(X_test,y_test)

preds, prediction_average = model.predict(X=X_train,y=y_train,theta=theta)    
prediction_average_list.append(prediction_average)

preds_test,prediction_average_test = model.predict(X_test,y_test,theta=theta)    
prediction_average_test_list.append(prediction_average_test)

cm = create_confusion_matrix(y_test, preds_test)

print("Probability averages on train: ",*prediction_average_list, sep =',')
print("Probability averages on test: ",*prediction_average_test_list, sep =',')
print("precision total:", precision_average(cm))
print("recall total:", recall_average(cm))   


print("label precision recall")
for label in range(10):
    print("label precision recall accuracy")
for label in range(10):
    print(f"{label:5d} {precision(label, cm):9.3f} {recall(label, cm):6.3f}")

/Users/danbridgman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:162: RuntimeWarning: divide by zero encountered in true_divide
/Users/danbridgman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in true_divide


Total number of records in X train: 29700
Total number of records in y train: 29700
Total number of features in X train: 784
Total number of records in X train (valdation set): 300
Total number of features in X train (validation set): 784
Total number of records in X test: 5000
Total number of features in X test: 784
#################################################
Logistic Regression model initialised with Max iterations = 500, K (classes) = 10, lmbda (Regularisation parameter) =1
#################################################
                                            
############################################
### Gradient Descent Optimisation beginning
############################################
Class 0 being optimised
Optimization terminated successfully.
         Current function value: 0.101262
         Iterations: 30
         Function evaluations: 65
         Gradient evaluations: 65
Class 1 being optimised
Optimization terminated successfully.
         Current function

In [ ]:
# Call the confusion matrix function to create a confusion matrix
# For either test or train
cm = create_confusion_matrix(y_train, preds)
#cm = create_confusion_matrix(y_test, preds_test)

fig, ax = plt.subplots()
fig.set_size_inches(8,8)

im = ax.imshow(cm, cmap='Oranges')

# Take the confusion matrix array, and make it pretty with colours and text :)

# A loop to go through every class and plot value for each actual vs pred class,
# with respective colour on the Yellow Green matplotlib colour scale
for i in range(len(np.unique(y_train))):
    for j in range(len(np.unique(y_train))):
        text = ax.text(j, i, cm[i, j],
                       ha="center", va="center", color="k")
#plt.title("Confusion matrix - Test Dataset", fontsize = 16)        
plt.title("Confusion matrix - Train Dataset", fontsize = 16)        

#plt.savefig("ConfusionMatrix_Train.png", dpi = 300)
plt.savefig("ConfusionMatrix_Test.png", dpi = 300)
plt.xlabel("Predicted classes ", fontsize = 16)   
plt.ylabel("Actual classes ", fontsize = 16)   
fig.tight_layout()        

plt.show()

In [ ]:
cm

In [ ]:

def calculate_acc_metrics(cm):
    tp = []
    tn = []
    fp = []
    fn = []
        
    for i in range(10):

        tp.append(cm[i][i])
        fp.append(sum(cm[:,i]) - cm[i][i])
        fn.append(sum(cm[i,:]) - cm[i][i])
        tn.append(np.sum(cm) - (tp + fp + fn))

    return(tp, fp, fn, tn)
    

    

        


In [ ]:
ACC = (TP+TN)/(TP+FP+FN+TN)
ACC

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

## Finally, output the predicted labels for the 5,000 test images we have

In [ ]:
final_predictions = model.final_predict(final_data_test, theta=theta[:,:-1])

with h5py.File('predicted_labels.h5','w') as H:
    H.create_dataset('label',data = final_predictions)
